## Steps to Implement the Real-Time Scoring Application
- Load the Trained Model: The ensemble model trained earlier will be serialized using joblib or pickle for deployment.
- Set Up the API Using Flask: Flask will handle incoming loan application requests, process them, and return a predicted default risk score.
- Data Preprocessing Pipeline: The preprocessing steps used in model training (e.g., encoding, scaling, feature engineering) will be applied to new loan applications before scoring.
- Generate Predictions: The API will take input features, transform them, and pass them to the model for inference.
- Return Results: The API will output the probability of loan default, allowing lenders to make data-driven decisions.

In [2]:
import pickle
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Load the trained model and preprocessor
with open("ensemble_model.pkl", "rb") as model_file:
    model = pickle.load(model_file)

with open("preprocessor.pkl", "rb") as preprocessor_file:
    preprocessor = pickle.load(preprocessor_file)

app = Flask(__name__)

@app.route("/predict", methods=["POST"])
def predict():
    """Receives loan application data and predicts default risk."""
    try:
        data = request.json
        df = pd.DataFrame([data])  # Convert input data to DataFrame
        
        # Apply preprocessing
        processed_data = preprocessor.transform(df)
        
        # Generate prediction
        prediction_prob = model.predict_proba(processed_data)[:, 1]
        prediction = (prediction_prob > 0.5).astype(int)
        
        return jsonify({
            "default_risk_probability": float(prediction_prob),
            "predicted_default_status": int(prediction)
        })
    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == "__main__":
    app.run(debug=True)


FileNotFoundError: [Errno 2] No such file or directory: 'ensemble_model.pkl'

## Usage Instructions
- Save the trained ensemble model and preprocessing pipeline using pickle.dump().
- Deploy the Flask API on a local server or cloud platform (e.g., AWS, Azure, or Google Cloud).
- Send loan application data via a POST request to /predict, formatted as JSON.
- Receive the predicted default risk score and default classification.